## `import duckdb` for Atlas Data Measurements analysis

Say, which one of these command you find more inteligible? 


1. Pandas Dataframe language:
 ```python
result = filtered_df.groupby('rdata').agg({'rt': 'mean', 'rdata': 'size'}).rename(columns={'rdata': 'responses'}).reset_index()
```

2. DuckDB syntax

```python
res=con.sql('''
select rdata,avg(rt),count(1) as responses from atlas where 
    rdata ~ 'b[0-9]-[A-Za-z]{3}'
    AND rdata IS NOT NULL
    group by rdata
''')
```


If you are often confused or irritated by `pandas` dataframe syntax, _suffer no more_. 

You can use SQL instead without any DB server. On your notebook.
And it's fast.

We showcase here [duckdb](https://duckdb.org/), an open-source  which can be used for Atlas measurement analysis.

Duckdb is:
   * Fully integrated with pandas (so you can import and export from and to)
   * Allows SQL execution on data 
   * A in-memory database
   * Severless (in python)


## Pandas vs Duckdb

In this example, we will compare how you can use `pandas` and `dudckdb` to some some analysis on Ripe Atlas measurements. 

The goal is to show a simple use case so you can use it later on your own analysis.


### Experiment

We will obtain a DNS data from Ripe Atlas and Atlas Probe information and perform some basic statistics


## Part 1: Data preparation

In [165]:
#setting the measurements we will analyze

#b-root atlas measrurements, `chaos queries`, 15min 
# see more on https://www.isi.edu/~johnh/PAPERS/Moura16b.pdf for more on these measurements

b_root_measurements="https://atlas.ripe.net/api/v2/measurements/10310/results/?start=1730025000&stop=1730025900&format=json"

#Altas probes metadata, daily compiled
probes_file="https://ftp.ripe.net/ripe/atlas/probes/archive/2024/10/20241027.json.bz2"

In [166]:
#import python libraries
import pandas as pd
import json
import duckdb
import requests

import bz2
from io import BytesIO 

In [167]:
b_root=requests.get(b_root_measurements)

In [168]:
b_root_data=b_root.json()

In [169]:
#sample data
b_root_data[0]

{'fw': 4790,
 'lts': 8,
 'dst_addr': '170.247.170.2',
 'af': 4,
 'src_addr': '192.168.178.26',
 'proto': 'UDP',
 'result': {'rt': 22.052,
  'size': 50,
  'abuf': 'aVeAAAABAAEAAAAACGhvc3RuYW1lBGJpbmQAABAAA8AMABAAAwAAAAAABwZiMS1hbXM=',
  'ID': 26967,
  'ANCOUNT': 1,
  'QDCOUNT': 1,
  'NSCOUNT': 0,
  'ARCOUNT': 0,
  'answers': [{'TYPE': 'TXT', 'NAME': 'hostname.bind', 'RDATA': ['b1-ams']}]},
 'msm_id': 10310,
 'prb_id': 1,
 'timestamp': 1730025017,
 'msm_name': 'Tdig',
 'from': '45.138.229.91',
 'type': 'dns',
 'stored_timestamp': 1730025074}

In [170]:
def extract_info(data):
    prb_id = data.get('prb_id')
    timestamp = data.get('timestamp')
    src_addr = data.get('src_addr')
    rt = data.get('result', {}).get('rt')
    answers = data.get('result', {}).get('answers', [{}])
    
    # Ensure 'answers' is a non-empty list and 'RDATA' is in the first item
    rdata = None
    if answers and 'RDATA' in answers[0]:
        rdata = answers[0]['RDATA'][0] if answers[0]['RDATA'] else 'None'

    return {
        'prb_id': prb_id,
        'timestamp': timestamp,
        'src_addr': src_addr,
        'rt': rt,
        'rdata': rdata
        
    }
    

In [171]:
data=[]
for k in b_root_data:
    data.append(extract_info(k))

In [172]:
print('We have retrieved ', len(data), ' measurements from Ripe Atlas')

We have retrieved  45781  measurements from Ripe Atlas


In [173]:
# now let's import it to a df
df = pd.DataFrame(data)


In [176]:
#create a new database
con=duckdb.connect('ripe')
# Create a table and insert the DataFrame
con.execute("CREATE TABLE IF NOT EXISTS atlas (prb_id INTEGER, timestamp INTEGER, src_addr TEXT, rt DOUBLE, rdata TEXT)")
con.execute("INSERT INTO atlas SELECT * FROM df")
# Query the table to verify
result = con.execute("SELECT * FROM atlas limit 10").fetchall()
print(result)



[(1, 1730025017, '192.168.178.26', 22.052, 'b1-ams'), (1000071, 1730025005, '192.168.11.78', 26.959, 'b3-ams'), (1000082, 1730025031, '10.104.28.189', 157.843, 'b4-mia'), (1000087, 1730025010, '192.168.1.254', 21.212, 'b3-ams'), (1000088, 1730025025, '10.50.84.12', 17.236, None), (1000090, 1730025008, '45.77.57.71', 5.702, 'b3-ams'), (1000093, 1730025055, '172.16.32.16', 18.574, 'b1-ams'), (1000095, 1730025041, '31.187.64.140', 13.046, 'b3-ams'), (1000096, 1730025038, '176.126.70.164', 20.167, 'b3-ams'), (1000143, 1730025006, '150.145.72.97', None, None)]


In [177]:
#let's create a probe list to create later a dataframe and a new table
## now we're going to download the probes metadata file
#ok, let's now get the probes metadata
# Download the file
response = requests.get(probes_file)
compressed_data = BytesIO(response.content)

# Decompress the data in memory
with bz2.BZ2File(compressed_data, 'rb') as file:
    probes = json.load(file)

print(probes['objects'][0])
    

pr_list=[]

for k in probes['objects']:
    pr_list.append(k)
    
#pandas df of probes
df_prb=pd.DataFrame(pr_list)

# now let's create  a duckdb table
# yeah, we use pandas for it :)

# Create a table and insert the DataFrame
con.execute("CREATE TABLE IF NOT EXISTS probes AS SELECT * FROM df_prb")

# Verify the data in the DuckDB table
test_query = con.execute("SELECT * FROM probes").fetchdf()


{'id': 1, 'address_v4': '45.138.229.91', 'address_v6': '2a10:3781:e22:1:220:4aff:fec8:23d7', 'asn_v4': 206238, 'asn_v6': 206238, 'prefix_v4': '45.138.228.0/22', 'prefix_v6': '2a10:3780::/29', 'is_anchor': False, 'is_public': True, 'status': 1, 'status_since': 1728711336, 'first_connected': 1288367583, 'total_uptime': 428217106, 'tags': ['dsl', 'home', 'nat', 'native-ipv6', 'ipv6', 'system-v1', 'system-ipv6-capable', 'system-ipv4-rfc1918', 'xs4all', 'system-ipv4-stable-1d', 'system-ipv4-works', 'system-ipv6-works', 'system-resolves-a-correctly', 'system-resolves-aaaa-correctly', 'system-ipv4-capable', 'system-ipv4-stable-30d'], 'country_code': 'NL', 'latitude': 52.3475, 'longitude': 4.9275, 'day': '20241027', 'probe': 'https://atlas.ripe.net/api/v2/probes/1/', 'status_name': 'Connected'}


# Parte 2: Data analysis

OK, so now we have two the same datasets (ripe atlas measurements and probes metadatata) stored in two different ways:
  * Pandas dataframes:
      * Measurements= `df`
      * probes's metadata= `df_prb`
  * Duckdb tables:
      * Measurements= `atlas`
      * probes's metadata= `probes`
  
Let's see how we can perform the same operations on both:

In [178]:
# let's start with how many queries we have
# let's compute the average RTT to each of the anycast server of b-root
# each site (each serfver location) is identified by the rdata field

# option 1: pandas

# Compute average rt for each rdata
average_rt = df.groupby('rdata')['rt'].mean().reset_index()

print(average_rt)



                                   rdata          rt
0                                   DNS1    0.724500
1                                   DNS2    2.280000
2                                   DNS3    1.031000
3                          DNS_AF_NH3_01    4.687000
4                          DNS_AF_NH3_13    4.401000
5                          DNS_AF_NH3_15    4.119000
6                          DNS_AF_NTH_16    4.109000
7                                    NS1    1.195000
8                     OPNsense.local.lan    1.421500
9                    OPNsense.stuyts.com    0.464750
10                    OPNsense.zunie.lan    0.467750
11                           alliance-45    1.789000
12                                     b   65.556000
13                                b1-ams   36.881509
14                                b1-iad   53.462525
15                                b1-lax  122.322482
16                                b1-mia  127.123686
17                                b1-scl   25.

### Deadling with bogus data

As shown above, there are many results, but let's filtered out the valid ones.
   * (Some DNS queires from Ripe Atlas clients are intercepted and hijacked, see [Moura16b](https://ant.isi.edu/~johnh/PAPERS/Moura16a.pdf) and [Nosyk23a](https://pure.tudelft.nl/ws/portalfiles/portal/151232870/978_3_031_28486_1_19.pdf) about it)
   
This means we need to filter out the data.

We can do it by looking at the `rdata` strings. For [B-ROOT](https://b.root-servers.org/), it has the follow format:
   *  bn-ABC , where `n` is a integer and `ABC` is a three letter string, denoting Airport Code
   
 
So let's see how can we do this filtering in both Pandas and Duckdb
   


In [180]:
# Filter rows where rdata matches the pattern bn-ABC
filtered_df = df[df['rdata'].str.match(r'b\d-[A-Za-z]{3}', na=False)]

# Compute average rt and count for each rdata
result = filtered_df.groupby('rdata').agg({'rt': 'mean', 'rdata': 'size'}).rename(columns={'rdata': 'responses'}).reset_index()

print(result)


     rdata          rt  responses
0   b1-ams   36.881509       9621
1   b1-iad   53.462525       1639
2   b1-lax  122.322482        465
3   b1-mia  127.123686        497
4   b1-scl   25.029500         60
5   b1-sin   70.688205        772
6   b2-iad   54.949259       1710
7   b2-lax  109.025563        467
8   b2-mia  127.239884        524
9   b2-scl   25.383507        144
10  b2-sin   72.801364        758
11  b3-ams   35.813374       9645
12  b3-iad   58.834110       1949
13  b3-lax  113.773521        484
14  b3-mia  114.486817        491
15  b3-scl   20.429663         83
16  b3-sin   79.691088        735
17  b4-ams   36.667410       9578
18  b4-iad   51.383624       1848
19  b4-lax  118.284348        443
20  b4-mia  135.924015        549
21  b4-sin   78.695698        884


In [181]:
#Ok, so how would you do it in duckdb
res=con.sql('''
select rdata,avg(rt),count(1) as responses from atlas where 
    rdata ~ 'b[0-9]-[A-Za-z]{3}'
    AND rdata IS NOT NULL
    group by rdata
''')

print(res)


┌─────────┬────────────────────┬───────────┐
│  rdata  │      avg(rt)       │ responses │
│ varchar │       double       │   int64   │
├─────────┼────────────────────┼───────────┤
│ b4-ams  │   36.6674102109001 │     28734 │
│ b4-sin  │  78.69569796380097 │      2652 │
│ b3-iad  │  58.83410979989737 │      5847 │
│ b3-sin  │  79.69108843537417 │      2205 │
│ b4-mia  │ 135.92401457194896 │      1647 │
│ b1-lax  │    122.32248172043 │      1395 │
│ b2-lax  │ 109.02556316916505 │      1401 │
│ b2-sin  │  72.80136411609499 │      2274 │
│ b2-mia  │  127.2398835877865 │      1572 │
│ b1-ams  │ 36.885662150455666 │     28859 │
│ b1-iad  │  53.46252471018906 │      4917 │
│ b4-iad  │  51.38362391774884 │      5544 │
│ b3-mia  │  114.4868167006109 │      1473 │
│ b3-ams  │  35.81337428719551 │     28935 │
│ b1-mia  │ 127.12368611670028 │      1491 │
│ b2-iad  │  54.94925906432744 │      5130 │
│ b3-lax  │ 113.77352066115692 │      1452 │
│ b1-sin  │  70.68820466321243 │      2316 │
│ b4-lax  

**Comment**: I personally find the SQL synthax more readable. 
(It's a matter of taste)

## Exercise 2: 

Shifting gears: the exercice now is to obtain the average RTT from the probes, to B-ROOT, based on its country of origin.

Note, however, that the country of origin is _not_ on the results table, but on the probe metadata.

So we need to join the results of two tables.

Let's first start with DuckDB

In [185]:
''' We can do like this on SQL

* We join the tables on r.prb_id = p.id, which is the Atlas probe ID
* We filter probes that reach B-ROOT servers by looking into `rdata` string
* We group by country_code

'''
res = con.sql('''
    SELECT
        p.country_code,
        AVG(r.rt) AS avg_rt,
        COUNT(r.rt) AS total_measurements,
        COUNT(DISTINCT r.prb_id) AS number_atlas_probes
    FROM
        atlas r
    JOIN
        probes p
    ON
        r.prb_id = p.id
    WHERE
        r.rdata ~ 'b[0-9]-[A-Za-z]{3}' AND r.rdata IS NOT NULL
    GROUP BY
        p.country_code
    ORDER BY
        avg_rt DESC;
''')



In [186]:
print(res)

┌──────────────┬────────────────────┬────────────────────┬─────────────────────┐
│ country_code │       avg_rt       │ total_measurements │ number_atlas_probes │
│   varchar    │       double       │       int64        │        int64        │
├──────────────┼────────────────────┼────────────────────┼─────────────────────┤
│ VU           │ 325.63949999999994 │                 12 │                   1 │
│ NC           │           290.8315 │                 48 │                   4 │
│ KI           │  272.4568571428571 │                 21 │                   2 │
│ MM           │ 263.70525000000004 │                 12 │                   1 │
│ NP           │ 247.75570454545453 │                132 │                  12 │
│ PK           │ 225.83114285714285 │                 21 │                   2 │
│ CI           │ 210.42600000000002 │                  3 │                   1 │
│ MW           │ 207.53987500000008 │                 24 │                   2 │
│ GQ           │ 206.7997500

In [187]:
# now, let's do the same for pandas

# Merge DataFrames on prb_id and id
merged_df = pd.merge(df, df_prb, left_on='prb_id', right_on='id')



# Filter rows where rdata matches the pattern bX-ABC and is not None
filtered_df = merged_df[merged_df['rdata'].notna() & merged_df['rdata'].str.match(r'b[0-9]-[A-Za-z]{3}')]

# Group by country_code and calculate the required metrics
result = filtered_df.groupby('country_code').agg(
    avg_rt=('rt', 'mean'),
    total_measurements=('rt', 'size'),
    number_atlas_probes=('prb_id', 'nunique')
).reset_index()

# Sort by avg_rt descending
result = result.sort_values(by='avg_rt', ascending=False)

print(result)

    country_code      avg_rt  total_measurements  number_atlas_probes
170           VU  325.639500                   4                    1
114           NC  290.831500                  16                    4
81            KI  272.456857                   7                    2
103           MM  263.705250                   4                    1
119           NP  247.755705                  44                   12
..           ...         ...                 ...                  ...
40            DE   20.819391                6003                 1636
54            GG   19.163000                   4                    1
14            BE   16.377820                 599                  160
94            LU   13.248532                 154                   41
117           NL   12.010847                2142                  575

[174 rows x 4 columns]


**Comment**: again, I just prefer to write SQL to do these sort of queries and complex queries




### Sanity check

In [188]:

#sanity check
test_pandas=result[result['country_code']=='ES']

print("results pandas" )
print(test_pandas.to_string(index=False))

test_duckdb=con.sql('''
SELECT
        p.country_code,
        AVG(r.rt) AS avg_rt,
        COUNT(r.rt) AS total_measurements,
        COUNT(DISTINCT r.prb_id) AS number_atlas_probes
    FROM
        atlas r
    JOIN
        probes p
    ON
        r.prb_id = p.id
    WHERE
        r.rdata ~ 'b[0-9]-[A-Za-z]{3}' AND r.rdata IS NOT NULL
    GROUP BY
        p.country_code
    HAVING
        p.country_code = 'ES'
    ORDER BY
        avg_rt DESC;

''')
print("results duckdb")
print(test_duckdb.to_df().to_string(index=False))


results pandas
country_code    avg_rt  total_measurements  number_atlas_probes
          ES 42.404476                 828                  221
results duckdb
country_code    avg_rt  total_measurements  number_atlas_probes
          ES 42.404476                2484                  221


In [190]:
## Extra

#1. You can export your sql results to Dataframes and carry on

res = con.sql('''
    SELECT
        p.country_code,
        AVG(r.rt) AS avg_rt,
        COUNT(r.rt) AS total_measurements,
        COUNT(DISTINCT r.prb_id) AS number_atlas_probes
    FROM
        atlas r
    JOIN
        probes p
    ON
        r.prb_id = p.id
    WHERE
        r.rdata ~ 'b[0-9]-[A-Za-z]{3}' AND r.rdata IS NOT NULL
    GROUP BY
        p.country_code
    ORDER BY
        avg_rt DESC;
''')

test_df=res.to_df()


In [191]:
print(test_df)

    country_code      avg_rt  total_measurements  number_atlas_probes
0             VU  325.639500                  12                    1
1             NC  290.831500                  48                    4
2             KI  272.456857                  21                    2
3             MM  263.705250                  12                    1
4             NP  247.755705                 132                   12
..           ...         ...                 ...                  ...
170           GG   19.163000                  12                    1
171           BE   16.377820                1797                  160
172           LU   13.248532                 462                   41
173           NL   12.010847                6426                  575
174         None    5.950500                  12                    1

[175 rows x 4 columns]


# Conclusion

* There's an alterantive to dataframe language
* Duckdb enables you to write clear SQL
* We have shown an alternative engine for analysis for improved clarity

